In [ ]:
!pip install tensorflow

     |████████████████████████████████| 462 kB 3.5 MB/s 


#BAB 5

5.1 Layer dan Blok

In [ ]:
import tensorflow as tf

net = tf.keras.models.Sequential([
    tf.keras.layers.Dense(256, activation=tf.nn.relu),
    tf.keras.layers.Dense(10),
])

X = tf.random.uniform((2, 20))
net(X)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[-0.2339084 , -0.06683626, -0.10363866, -0.03862647, -0.40292042,
        -0.03054691,  0.02197126, -0.16653362,  0.28527445,  0.29226616],
       [-0.17525145, -0.02505462, -0.154076  , -0.19030795, -0.288176  ,
         0.0615394 , -0.14093678, -0.24503973,  0.33942378,  0.16297255]],
      dtype=float32)>

In [ ]:
class MLP(tf.keras.Model):
    # Declare a layer with model parameters. Here, we declare two fully
    # connected layers
    def __init__(self):
        # Call the constructor of the `MLP` parent class `Model` to perform
        # the necessary initialization. In this way, other function arguments
        # can also be specified during class instantiation, such as the model
        # parameters, `params` (to be described later)
        super().__init__()
        # Hidden layer
        self.hidden = tf.keras.layers.Dense(units=256, activation=tf.nn.relu)
        self.out = tf.keras.layers.Dense(units=10)  # Output layer

    # Define the forward propagation of the model, that is, how to return the
    # required model output based on the input `X`
    def call(self, X):
        return self.out(self.hidden((X)))

In [ ]:
net = MLP()
net(X)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[ 0.2139617 , -0.31394175, -0.06348141,  0.1917887 ,  0.02598595,
         0.06970533,  0.20106244, -0.23791005,  0.05406186, -0.07088925],
       [ 0.12727495, -0.44473487,  0.18613918,  0.27770627,  0.18234487,
         0.228347  ,  0.36232072, -0.01694701,  0.17893237,  0.03861136]],
      dtype=float32)>

In [ ]:
class MySequential(tf.keras.Model):
    def __init__(self, *args):
        super().__init__()
        self.modules = []
        for block in args:
            # Here, `block` is an instance of a `tf.keras.layers.Layer`
            # subclass
            self.modules.append(block)

    def call(self, X):
        for module in self.modules:
            X = module(X)
        return X

In [ ]:
net = MySequential(
    tf.keras.layers.Dense(units=256, activation=tf.nn.relu),
    tf.keras.layers.Dense(10))
net(X)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[ 0.06615081, -0.01881247,  0.08252638,  0.20262164, -0.07634783,
        -0.65316546,  0.46824265,  0.11830381, -0.2836047 ,  0.1827244 ],
       [-0.1331569 , -0.09514139, -0.10812029,  0.22354689, -0.43269986,
        -0.62358904,  0.59553087,  0.02787521, -0.34621406,  0.08602829]],
      dtype=float32)>

In [ ]:
class FixedHiddenMLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()
        # Random weight parameters created with `tf.constant` are not updated
        # during training (i.e., constant parameters)
        self.rand_weight = tf.constant(tf.random.uniform((20, 20)))
        self.dense = tf.keras.layers.Dense(20, activation=tf.nn.relu)

    def call(self, inputs):
        X = self.flatten(inputs)
        # Use the created constant parameters, as well as the `relu` and
        # `matmul` functions
        X = tf.nn.relu(tf.matmul(X, self.rand_weight) + 1)
        # Reuse the fully-connected layer. This is equivalent to sharing
        # parameters with two fully-connected layers
        X = self.dense(X)
        # Control flow
        while tf.reduce_sum(tf.math.abs(X)) > 1:
            X /= 2
        return tf.reduce_sum(X)

In [ ]:
net = FixedHiddenMLP()
net(X)

<tf.Tensor: shape=(), dtype=float32, numpy=0.74393207>

In [ ]:
class NestMLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.net = tf.keras.Sequential()
        self.net.add(tf.keras.layers.Dense(64, activation=tf.nn.relu))
        self.net.add(tf.keras.layers.Dense(32, activation=tf.nn.relu))
        self.dense = tf.keras.layers.Dense(16, activation=tf.nn.relu)

    def call(self, inputs):
        return self.dense(self.net(inputs))

chimera = tf.keras.Sequential()
chimera.add(NestMLP())
chimera.add(tf.keras.layers.Dense(20))
chimera.add(FixedHiddenMLP())
chimera(X)

<tf.Tensor: shape=(), dtype=float32, numpy=0.69162375>

#5.2 Parameter management

In [ ]:
import tensorflow as tf

net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(4, activation=tf.nn.relu),
    tf.keras.layers.Dense(1),
])

X = tf.random.uniform((2, 4))
net(X)

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[-0.7200342 ],
       [-0.66909397]], dtype=float32)>

5.2.1 Parameter Access

In [ ]:
print(net.layers[2].weights)

[<tf.Variable 'dense_21/kernel:0' shape=(4, 1) dtype=float32, numpy=
array([[-0.98597205],
       [-0.892779  ],
       [-0.4502076 ],
       [-0.00477719]], dtype=float32)>, <tf.Variable 'dense_21/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>]


5.2.1.1 Targeted Parameter

In [ ]:
print(type(net.layers[2].weights[1]))
print(net.layers[2].weights[1])
print(tf.convert_to_tensor(net.layers[2].weights[1]))

<class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'>
<tf.Variable 'dense_21/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>
tf.Tensor([0.], shape=(1,), dtype=float32)


5.2.1.2 All parameter at once

In [ ]:
print(net.layers[1].weights)
print(net.get_weights())

[<tf.Variable 'dense_20/kernel:0' shape=(4, 4) dtype=float32, numpy=
array([[-0.2661016 , -0.4309085 , -0.6076398 , -0.7835085 ],
       [ 0.86100155, -0.49471223, -0.7049935 ,  0.8002412 ],
       [ 0.25713664, -0.28915304,  0.06006652,  0.5826686 ],
       [ 0.8040826 , -0.28175336,  0.8114807 ,  0.09785897]],
      dtype=float32)>, <tf.Variable 'dense_20/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>]
[array([[-0.2661016 , -0.4309085 , -0.6076398 , -0.7835085 ],
       [ 0.86100155, -0.49471223, -0.7049935 ,  0.8002412 ],
       [ 0.25713664, -0.28915304,  0.06006652,  0.5826686 ],
       [ 0.8040826 , -0.28175336,  0.8114807 ,  0.09785897]],
      dtype=float32), array([0., 0., 0., 0.], dtype=float32), array([[-0.98597205],
       [-0.892779  ],
       [-0.4502076 ],
       [-0.00477719]], dtype=float32), array([0.], dtype=float32)]


In [ ]:
net.get_weights()[1]

array([0., 0., 0., 0.], dtype=float32)

5.2.1.3 Collecting Parameters from nested block

In [ ]:
def block1(name):
    return tf.keras.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(4, activation=tf.nn.relu)],
        name=name)

def block2():
    net = tf.keras.Sequential()
    for i in range(4):
        # Nested here
        net.add(block1(name=f'block-{i}'))
    return net

rgnet = tf.keras.Sequential()
rgnet.add(block2())
rgnet.add(tf.keras.layers.Dense(1))
rgnet(X)

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[-0.00036203],
       [-0.00017317]], dtype=float32)>

In [ ]:
print(rgnet.summary())

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_8 (Sequential)   (2, 4)                    80        
                                                                 
 dense_26 (Dense)            (2, 1)                    5         
                                                                 
Total params: 85
Trainable params: 85
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
rgnet.layers[0].layers[1].layers[1].weights[1]

<tf.Variable 'dense_23/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>

5.2.2 Parameter inilization 

misalkan saya weightnya itu 1

In [ ]:
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(
        4, activation=tf.nn.relu,
        kernel_initializer=tf.random_normal_initializer(mean=0, stddev=0.01),
        bias_initializer=tf.zeros_initializer()),
    tf.keras.layers.Dense(1)])

net(X)
net.weights[0], net.weights[1]

(<tf.Variable 'dense_27/kernel:0' shape=(4, 4) dtype=float32, numpy=
 array([[ 0.00434056, -0.00540833, -0.02315391, -0.005995  ],
        [-0.00195489, -0.00948973,  0.00029624,  0.00755205],
        [ 0.01071004,  0.00835577,  0.0050816 ,  0.00473668],
        [-0.01115704, -0.01005614, -0.00348999,  0.00520753]],
       dtype=float32)>,
 <tf.Variable 'dense_27/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>)

In [ ]:
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(
        4,
        activation=tf.nn.relu,
        kernel_initializer=tf.keras.initializers.GlorotUniform()),
    tf.keras.layers.Dense(
        1, kernel_initializer=tf.keras.initializers.Constant(42)),
])

net(X)
print(net.layers[1].weights[0])
print(net.layers[2].weights[0])

<tf.Variable 'dense_29/kernel:0' shape=(4, 4) dtype=float32, numpy=
array([[-0.54799414, -0.39163086,  0.48127037,  0.37723476],
       [ 0.11593616, -0.27532512,  0.7319192 , -0.84172726],
       [ 0.7212979 , -0.54171   ,  0.57707304,  0.76227254],
       [-0.4884279 ,  0.4822132 ,  0.3731634 , -0.2678554 ]],
      dtype=float32)>
<tf.Variable 'dense_30/kernel:0' shape=(4, 1) dtype=float32, numpy=
array([[42.],
       [42.],
       [42.],
       [42.]], dtype=float32)>


5.2.2.2 Custom Inilization

In [ ]:
class MyInit(tf.keras.initializers.Initializer):
    def __call__(self, shape, dtype=None):
        data=tf.random.uniform(shape, -10, 10, dtype=dtype)
        factor=(tf.abs(data) >= 5)
        factor=tf.cast(factor, tf.float32)
        return data * factor

net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(
        4,
        activation=tf.nn.relu,
        kernel_initializer=MyInit()),
    tf.keras.layers.Dense(1),
])

net(X)
print(net.layers[1].weights[0])

<tf.Variable 'dense_31/kernel:0' shape=(4, 4) dtype=float32, numpy=
array([[-6.5876102,  0.       , -0.       ,  0.       ],
       [ 6.4443054,  0.       ,  6.455927 ,  0.       ],
       [ 9.408459 , -6.152625 , -5.410857 ,  6.22847  ],
       [-0.       , -9.002363 , -9.731419 , -0.       ]], dtype=float32)>


In [ ]:
net.layers[1].weights[0][:].assign(net.layers[1].weights[0] + 1)
net.layers[1].weights[0][0, 0].assign(42)
net.layers[1].weights[0]

<tf.Variable 'dense_31/kernel:0' shape=(4, 4) dtype=float32, numpy=
array([[42.       ,  1.       ,  1.       ,  1.       ],
       [ 7.4443054,  1.       ,  7.455927 ,  1.       ],
       [10.408459 , -5.152625 , -4.410857 ,  7.22847  ],
       [ 1.       , -8.002363 , -8.731419 ,  1.       ]], dtype=float32)>

5.2.3 Tied Parameter 

In [ ]:
# tf.keras behaves a bit differently. It removes the duplicate layer
# automatically
shared = tf.keras.layers.Dense(4, activation=tf.nn.relu)
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    shared,
    shared,
    tf.keras.layers.Dense(1),
])

net(X)
# Check whether the parameters are different
print(len(net.layers) == 3)

True


#5.3 Deferred Initialization 

5.3.1 Instantiating Network 

In [ ]:
import tensorflow as tf

net = tf.keras.models.Sequential([
    tf.keras.layers.Dense(256, activation=tf.nn.relu),
    tf.keras.layers.Dense(10),
])

In [ ]:
[net.layers[i].get_weights() for i in range(len(net.layers))]

[[], []]

In [ ]:
X = tf.random.uniform((2, 20))
net(X)
[w.shape for w in net.get_weights()]

[(20, 256), (256,), (256, 10), (10,)]

#5.4 Custom Layers 
5.4.1 Layers without parameters 

In [ ]:
import tensorflow as tf


class CenteredLayer(tf.keras.Model):
    def __init__(self):
        super().__init__()

    def call(self, inputs):
        return inputs - tf.reduce_mean(inputs)

In [ ]:
layer = CenteredLayer()
layer(tf.constant([1, 2, 3, 4, 5]))

<tf.Tensor: shape=(5,), dtype=int32, numpy=array([-2, -1,  0,  1,  2], dtype=int32)>

In [ ]:
net = tf.keras.Sequential([tf.keras.layers.Dense(128), CenteredLayer()])

In [ ]:
Y = net(tf.random.uniform((4, 8)))
tf.reduce_mean(Y)

<tf.Tensor: shape=(), dtype=float32, numpy=-1.1641532e-09>

5.4.2 Layer with Parameter 

In [ ]:
class MyDense(tf.keras.Model):
    def __init__(self, units):
        super().__init__()
        self.units = units

    def build(self, X_shape):
        self.weight = self.add_weight(name='weight',
            shape=[X_shape[-1], self.units],
            initializer=tf.random_normal_initializer())
        self.bias = self.add_weight(
            name='bias', shape=[self.units],
            initializer=tf.zeros_initializer())

    def call(self, X):
        linear = tf.matmul(X, self.weight) + self.bias
        return tf.nn.relu(linear)

In [ ]:
dense = MyDense(3)
dense(tf.random.uniform((2, 5)))
dense.get_weights()

[array([[-0.05521664, -0.04526592, -0.05158228],
        [-0.04883691, -0.01863797,  0.07063188],
        [-0.03218503,  0.04535903,  0.02856409],
        [-0.01939636, -0.02840298,  0.02222984],
        [-0.07779368,  0.00041903,  0.03801968]], dtype=float32),
 array([0., 0., 0.], dtype=float32)]

In [ ]:
dense(tf.random.uniform((2, 5)))

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[0.        , 0.        , 0.05297275],
       [0.        , 0.00503999, 0.05022965]], dtype=float32)>

In [ ]:
net = tf.keras.models.Sequential([MyDense(8), MyDense(1)])
net(tf.random.uniform((2, 64)))

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[0.02010227],
       [0.01113593]], dtype=float32)>

5.5 File I/O
5.5.1 Loading and saving tensors 

In [ ]:
import numpy as np
import tensorflow as tf

x = tf.range(4)
np.save('x-file.npy', x)

In [ ]:
x2 = np.load('x-file.npy', allow_pickle=True)
x2

array([0, 1, 2, 3], dtype=int32)

In [ ]:
y = tf.zeros(4)
np.save('xy-files.npy', [x, y])
x2, y2 = np.load('xy-files.npy', allow_pickle=True)
(x2, y2)

(array([0., 1., 2., 3.]), array([0., 0., 0., 0.]))

In [ ]:
mydict = {'x': x, 'y': y}
np.save('mydict.npy', mydict)
mydict2 = np.load('mydict.npy', allow_pickle=True)
mydict2

array({'x': <tf.Tensor: shape=(4,), dtype=int32, numpy=array([0, 1, 2, 3], dtype=int32)>, 'y': <tf.Tensor: shape=(4,), dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>},
      dtype=object)

5.5.2 Loading and saving model parameter 

In [ ]:
class MLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.hidden = tf.keras.layers.Dense(units=256, activation=tf.nn.relu)
        self.out = tf.keras.layers.Dense(units=10)

    def call(self, inputs):
        x = self.flatten(inputs)
        x = self.hidden(x)
        return self.out(x)

net = MLP()
X = tf.random.uniform((2, 20))
Y = net(X)

In [ ]:
net.save_weights('mlp.params')

In [ ]:
clone = MLP()
clone.load_weights('mlp.params')

In [ ]:
Y_clone = clone(X)
Y_clone == Y

<tf.Tensor: shape=(2, 10), dtype=bool, numpy=
array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True]])>

5.6 GPU's